In [ ]:
# default_exp evaluation.metrics

# Metrics
> Metrics.

In [ ]:
#hide
from nbdev.showdoc import *
from fastcore.nb_imports import *
from fastcore.test import *

In [ ]:
#export
import torch

In [ ]:
#export
def NDCG(true, pred):
    match = pred.eq(true).nonzero(as_tuple=True)[1]
    ncdg = torch.log(torch.Tensor([2])).div(torch.log(match + 2))
    ncdg = ncdg.sum().div(pred.shape[0]).item()
    return ncdg


def APAK(true, pred):
    k = pred.shape[1]
    apak = pred.eq(true).div(torch.arange(k) + 1)
    apak = apak.sum().div(pred.shape[0]).item()
    return apak


def HR(true, pred):
    hr = pred.eq(true).sum().div(pred.shape[0]).item()
    return hr


def get_eval_metrics(scores, true, k=10):
    test_items = [torch.LongTensor(list(item_scores.keys())) for item_scores in scores]
    test_scores = [torch.Tensor(list(item_scores.values())) for item_scores in scores]
    topk_indices = [s.topk(k).indices for s in test_scores]
    topk_items = [item[idx] for item, idx in zip(test_items, topk_indices)]
    pred = torch.vstack(topk_items)
    ncdg = NDCG(true, pred)
    apak = APAK(true, pred)
    hr = HR(true, pred)

    return ncdg, apak, hr

In [ ]:
scores = [{1: 0.2, 2: 0.3, 3: 0.4, 4: 0.5, 9: 0.1},
          {1: 0.2, 2: 0.3, 3: 0.4, 4: 0.5, 9: 0.1},
          {1: 0.2, 2: 0.3, 3: 0.4, 4: 0.5, 9: 0.1},
          {1: 0.2, 2: 0.3, 3: 0.4, 4: 0.5, 9: 0.1},
          {1: 0.2, 2: 0.3, 3: 0.4, 4: 0.5, 9: 0.1}]

true = torch.tensor([[1],[1],[2],[3],[4]])
metric = get_eval_metrics(scores, true, k=3)
metric

(0.4261859357357025, 0.36666667461395264, 0.6000000238418579)

In [ ]:
# it should all 1, because all relevant items are in range k=3
true = torch.tensor([[4],[4],[4],[4],[4]])
metric = get_eval_metrics(scores, true, k=3)
metric

(1.0, 1.0, 1.0)

In [ ]:
# it should all 0, because no relevant item is in range k=3
true = torch.tensor([[9],[1],[9],[1],[1]])
metric = get_eval_metrics(scores, true, k=3)
metric

(0.0, 0.0, 0.0)

> **References:-**
- https://github.com/massquantity/DBRL/blob/master/dbrl/evaluate/metrics.py
- [https://github.com/NVIDIA-Merlin/Transformers4Rec/blob/main/transformers4rec/torch/ranking_metric.py](https://github.com/NVIDIA-Merlin/Transformers4Rec/blob/main/transformers4rec/torch/ranking_metric.py)
- [https://github.com/karlhigley/ranking-metrics-torch](https://github.com/karlhigley/ranking-metrics-torch)
- [https://github.com/mquad/sars_tutorial/blob/master/util/metrics.py](https://github.com/mquad/sars_tutorial/blob/master/util/metrics.py)

In [ ]:
#hide
%reload_ext watermark
%watermark -a "Sparsh A." -m -iv -u -t -d -p recohut

Author: Sparsh A.

Last updated: 2022-01-06 09:02:26

recohut: 0.0.9

Compiler    : GCC 7.5.0
OS          : Linux
Release     : 5.4.144+
Machine     : x86_64
Processor   : x86_64
CPU cores   : 2
Architecture: 64bit

torchmetrics: 0.6.2
numpy       : 1.19.5
torch       : 1.10.0+cu111
PIL         : 7.1.2
matplotlib  : 3.2.2
IPython     : 5.5.0

